# PhotoFiTT: Cell activity analysis

The data is assumed to be organised in folders as follows. The Subcategories are the ones that the code uses to display the data. The labels will be identical to the folder name, so the same conditions should always have the same folder name.

   ```   
   -Raw-images (folder)
    |
    |--Biological-replica-date-1 (folder) [Subcaegory-00]
        |
        |--Cell density / UV Ligth / WL 475 light [Subcategory-01] 
           |
           |-- control-condition (folder) [Subcategory-02] 
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
           |
           |-- condition1 (folder) [Subcategory-02] 
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
           |
           |-- condition2 (folder) [Subcategory-02] 
           |    |  file1.tif
           |    |  file2.tif
           |    |  ...
        |
        |--Cell density / UV Ligth / WL 475 light [Subcategory-01]
        ...
   ```

## 1. Import required dependencies

In [5]:
import os
import numpy as np
import pandas as pd
import photofitt
from photofitt import display
from photofitt.analysis import extract_activity, cumulative_activity, estimate_proportional_deviations
from photofitt.utils import power_conversion, numerical_dose
import seaborn as sns
import matplotlib.pyplot as plt

## 2. Set the paths to the data and the output folders
- `path2images`: The path where the Raw-images are saved. For example "/home/ocb/HardDrive_4TB/PHX/SYNCHRO/Raw-images"
- `path2results`: The path where the notebook will save all the results. Once the cell activity is calculated, a CSV file together with the plots will be stored there.
- `frame_rate`: The time frame in the videos given in minutes (e.g., `frame_rate = 4`)


In [6]:
path2images = "/Users/esti/Library/CloudStorage/OneDrive-igc.gulbenkian.pt/Manuscript - PhotoFiTT/Manuscript - PhotoFiTT/PAPER/example-data-github/example-data-notebooks/downsample_data"
path2results = "/Users/esti/Documents/PROYECTOS/PHX/PhotoFiTT/example-data/cell-activity/"
frame_rate = 4 

## Prepare the directory to save the results
os.makedirs(path2results,exist_ok=True)
os.makedirs(os.path.join(path2results, "plots"), exist_ok=True)


## 3. Calculate cell activity.
This process can take long hours depending on the number of videos to analyse. If it is already calculated, do not run this cell again and jump to the next step

In [ ]:
activity_metrics = extract_activity(path2images,
                                    method="intensity",
                                    save_steps=False,
                                    enhance_contrast=True,
                                    output_path=path2results,
                                    frame_rate=frame_rate,
                                    condition=None)

## Save the results
activity_metrics.to_csv(os.path.join(path2results, "data_activity_intensity.csv"))

['.DS_Store', '2022-09-09-day', '20221014', '2022-09-08-day', '20230505_475_day', '2022-09-08-night', '2022-08-09']
['WL 630 - high density', '.DS_Store']
['6 jcm2', '.DS_Store', '0 jcm2']
['.DS_Store', 'CHO_630_live-02-Scene-42-P9-B04.tif', 'CHO_630_live-02-Scene-43-P6-B04.tif', 'CHO_630_live-02-Scene-41-P3-B04.tif']
CHO_630_live-02-Scene-42-P9-B04.tif
CHO_630_live-02-Scene-43-P6-B04.tif
CHO_630_live-02-Scene-41-P3-B04.tif
['CHO_630_live-02-Scene-03-P2-A01.tif', '.DS_Store', 'CHO_630_live-02-Scene-01-P9-A01.tif', 'CHO_630_live-02-Scene-02-P3-A01.tif']
CHO_630_live-02-Scene-03-P2-A01.tif
CHO_630_live-02-Scene-01-P9-A01.tif
CHO_630_live-02-Scene-02-P3-A01.tif
['WL 630 - high density', '.DS_Store']
['60 jcm2', '6 jcm2', '.DS_Store', '0 jcm2']
['CHO_630-live-02-Scene-31-P7-A04.tif', '.DS_Store', 'CHO_630-live-02-Scene-32-P1-A04.tif', 'CHO_630-live-02-Scene-33-P8-A04.tif']
CHO_630-live-02-Scene-31-P7-A04.tif
CHO_630-live-02-Scene-32-P1-A04.tif
CHO_630-live-02-Scene-33-P8-A04.tif
['CHO_630-

## 4. Analyse cell activity


### 4.1 Inspect the cell activity accross time
- `hue`:
- `hue_order`:
- `y_lim`:
- `format_extension`:

In [ ]:
activity_metrics = pd.read_csv(os.path.join(path2results, "data_activity_intensity.csv"))

# If the data was organised as indicated, Subcategory-01 corresponds to the wavelength (e.g., 385nm illumination)
wavelength = np.unique(activity_metrics["Subcategory-01"])

hue="Subcategory-02" # It's the variable that corresponds to the light dose (e.g., 60 J/cm2)
hue_order = ['0 jcm2','0.6 jcm2', '6 jcm2', '60 jcm2'] # Establish your own order if needed.
ylim = [0.0001, 0.0035] # This is the y-axis limit to plot everything similarly. Addapt it as needed. 
format_extension = 'pdf' # Format to save the plots.

for w in wavelength:
    activity_w = activity_metrics[activity_metrics["Subcategory-01"]==w].reset_index(drop=True)    

    # Plot the temporal distribution of the activity
    y = "mean activity" # Variable to display
    display.conditions_with_aggregates(activity_w, y,
                               f"Cell activity variance {w}",
                               os.path.join(path2results, "plots"),
                               f"activity_variance_{w}.png",
                               hue=hue, 
                               hue_order=hue_order,
                               ylim = ylim)
    
    ## Plot additive activity
    activity_w_dataframe, activity_w = cumulative_activity(activity_w, y, use_starting_point="None", starting_point=0)
    
    y_var = f"Cumulative {y}"
    name = "cell_cumulative_activity_{0}.{1}".format(w, format_extension)
    title = f"{w}"
    display.conditions(activity_w, y_var, 
                       title, 
                       hue,
                       os.path.join(path2results, "plots"),
                       name, 
                       hue_order=hue_order,
                       palette=sns.color_palette("CMRmap_r", len(hue_order)), # change the palette if desired
                       figsize=(4, 5)) # Change the aspect ratio of the figure if desired 
    

### 4.2 Display the cummulative cell activity at a specific timepoint
- `t`: The time-point at which the differences in activity will be evaluated, in the same unit as the frame_rate given (e.g., minutes)
- `variable`: The variable for which the proportional differences will be calculated. In our case it was `'Cumulative mean activity'`.
- `grouping_variable`: The differences are calculated on average by replica (the grouping variable). In our case, following the data structure, it should be `"Subcategory-00"`.
- `reference_variable`: The variable in the data-frame that encodes the conditions and that will be displayed. In our case, it is `"Subcategory-02"`.
- `reference_category`: The reference condition used to calculate the differences. In our case, it is the control `'0 J/cm2'`.
- `unique_id_var`: We use it to identify individual videos and get statistics across them. In our case, it is `"video_name"`.
- `add_columns`: List of the columns from the original dataset that we want to preserve in the new `activity_diff` data frame.

In [ ]:
t = 420
activity_metrics = pd.read_csv(os.path.join(path2results, "data_activity_intensity.csv"))
## Calculate the cumulative cell activity
y = "mean activity"
activity_dataframe, activity = cumulative_activity(activity_metrics, y, use_starting_point="None")
### Filter out the data for a specfic timepoint t.
activity = activity.loc[lambda activity: activity["frame"]==t]

variable = 'Cumulative mean activity'
grouping_variable="Subcategory-00"
reference_category='0 jcm2'
reference_variable="Subcategory-02"
unique_id_var = "video_name"
add_columns = ["Subcategory-01"]
## Calculate the deviations in cell activity
activity_diff = estimate_proportional_deviations(activity,
                                                 variable,
                                                 reference_category=reference_category,
                                                 reference_variable=reference_variable,
                                                 unique_id_var=unique_id_var,
                                                 grouping_variable=grouping_variable,
                                                 add_columns=add_columns) ## add




palette_colours = ['#BC77F8', '#30C7A6', '#FF7848'] # colors for the wavelengths
wl_order = ['WL UV - high density', 'WL 475 - high density', 'WL 630 - high density'] # Column order in the plot, corresponding to the wavelengths
wl_column = "Subcategory-01" # Name of the category to consider when plotting the columns.
hue_order = ['0 jcm2', '0.6 jcm2', '6 jcm2', '60 jcm2'] # Order in which the conditions will be displayed.
y_lim = [-25, 100] # Choose the y-axis limits in the plot
format_extension = "pdf" # Choose between "pdf" and "png"

# PLOT
g = sns.catplot(
    activity_diff, kind="bar",
    x=reference_variable, 
    y=f"proportional_difference_per_{unique_id_var}",
    col=wl_column,
    hue=wl_column,
    order=hue_order,
    col_order=wl_order,
    palette=palette_colours,
    errorbar=("ci", 95),  capsize=.08, linewidth=0.5,
    hue_order=wl_order, legend=False
)
g.map_dataframe(sns.stripplot,
            x=reference_variable,
            y=f"proportional_difference_per_{unique_id_var}",
            order=hue_order,
            jitter=True,
            dodge=False,
            marker='.',
            color="gray",
            alpha=1)
plt.tight_layout()
g.fig.get_axes()[-1].legend(loc='upper right')
g.set(xlabel="Time [min]", ylabel=f"proportional difference [t={t}min]", ylim=y_lim)
g.set_titles("{col_name}")
g.savefig(os.path.join(path2results, "plots", f"barplots_{variable}_differences_t{t}.{format_extension}"), format=format_extension, transparent=True)
plt.show()


### 4.3 Obtain basic statistics


#### Mean values

In [ ]:
activity_diff.groupby(["Subcategory-01", "Subcategory-02"]).mean()


#### Standard deviation values

In [ ]:
print("Standard deviation values of all the calculated metrics")
activity_diff.groupby(["Subcategory-01", "Subcategory-02"]).std()